In [2]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding,GRU,Dense,AdditiveAttention,StringLookup,TextVectorization
from tensorflow.keras.losses import SparseCategoricalCrossentropy
import csv

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

with open("/content/drive/MyDrive/data/eng-ind.csv") as f:
  csv_reader = csv.reader(f)
  next(csv_reader)

  input_data = list()
  target_data = list()

  for row in csv_reader:
    input_data.append(row[0])
    target_data.append(row[1])

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
BUFFER_SIZE = len(input_data)
BATCH_SIZE = 64

dataset = tf.data.Dataset.from_tensor_slices((input_data, target_data)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE)

In [ ]:
for input_batch, target_batch in dataset.take(1):
  print(input_batch,"\n")
  print(target_batch)
  break

In [ ]:
def tf_lower_and_split_punct(text):
  text = tf.strings.lower(text)
  text = tf.strings.regex_replace(text, '[^ a-z.?!,¿]', '')
  text = tf.strings.regex_replace(text, '[.?!,¿]', r' \0 ')
  text = tf.strings.strip(text)

  text = tf.strings.join(['[START]', text, '[END]'], separator=' ')
  return text

In [ ]:
max_vocab_size = 5000

input_text_processor = TextVectorization(
    standardize=tf_lower_and_split_punct,
    max_tokens=max_vocab_size)

output_text_processor = TextVectorization(
    standardize=tf_lower_and_split_punct,
    max_tokens=max_vocab_size)

In [ ]:
input_text_processor.adapt(input_data)
output_text_processor.adapt(target_data)

In [18]:
# ala ala aja ni
inputs = tf.keras.Input(shape=(3,))
x = tf.keras.layers.Dense(4, activation=tf.nn.relu)(inputs)
outputs = tf.keras.layers.Dense(5, activation=tf.nn.softmax)(x)

vocab_size = 5000
enc_units = 10
embedding_dim = 64

# encoding layer
embedding_enc = Embedding(vocab_size, embedding_dim)(inputs)
output_enc, state_enc = GRU(
    enc_units,return_sequences=True,
    return_state=True,
    recurrent_initializer='glorot_uniform')(embedding_enc)

model = tf.keras.Model(inputs=inputs, outputs=outputs)


In [10]:
model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 3)]               0         
                                                                 
 dense_2 (Dense)             (None, 4)                 16        
                                                                 
 dense_3 (Dense)             (None, 5)                 25        
                                                                 
Total params: 41
Trainable params: 41
Non-trainable params: 0
_________________________________________________________________
